In [14]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [15]:
import numpy as np
import cv2
from scipy.special import eval_chebyt
import matplotlib.pyplot as plt

In [16]:
def convert_to_polar(image):
    """
    Convert the input image from Cartesian (x, y) coordinates to polar (r, θ) coordinates.
    
    Parameters:
        image (2D numpy array): Grayscale image.
    
    Returns:
        polar_image (2D numpy array): The image in polar coordinates.
        r_coords (1D numpy array): The radial distances for each pixel.
        theta_coords (1D numpy array): The angular coordinates for each pixel.
    """
    rows, cols = image.shape
    center = (cols // 2, rows // 2)  # Center of the image

    # Create coordinate grids (X, Y) in Cartesian space
    x = np.linspace(-center[0], center[0], cols)
    y = np.linspace(-center[1], center[1], rows)
    X, Y = np.meshgrid(x, y)
    
    # Convert to polar coordinates
    r = np.sqrt(X**2 + Y**2)
    theta = np.arctan2(Y, X)
    
    # Normalize r to the range [0, 1]
    r = r / np.max(r)
    
    return image, r, theta

In [17]:
def radial_basis_function(n, t, r):
    """
    Compute the radial basis function R_t^n(r) for the fractional Chebyshev-Fourier moments.
    
    Parameters:
        n (int): Order of the Chebyshev polynomial.
        t (float): Fractional parameter.
        r (2D numpy array): Radial coordinates of the image.
    
    Returns:
        Rtn (2D numpy array): The computed radial basis function values.
    """
    Rtn = np.zeros_like(r)
    
    for k in range(n//2 + 1):
        Rtn += (-1)**k * (np.math.factorial(n - k) /
                          (np.math.factorial(k) * np.math.factorial(n - 2*k))) * \
                (4 * r**t - 2)**(n - 2*k)
    
    # Multiply by the fractional part
    Rtn *= np.sqrt(t) * r**(t - 1)
    
    return Rtn

In [18]:
def compute_weighted_amplitude(frchfm, tau=1, xi=1):
    """
    Compute the Weighted Amplitude (WA) matrix from the Fractional Chebyshev-Fourier Moments (FrCHFM).

    Parameters:
        frchfm (2D numpy array): Fractional Chebyshev-Fourier moments matrix (complex values).
        tau (int): Control parameter for order weight (default is 1).
        xi (int): Control parameter for repetition weight (default is 1).

    Returns:
        wa_matrix (2D numpy array): Weighted Amplitude matrix.
    """
    # Initialize weighted amplitude matrix with zeros
    wa_matrix = np.zeros_like(frchfm, dtype=float)

    # Calculate weighted amplitudes
    for n in range(frchfm.shape[0]):
        for m in range(frchfm.shape[1]):
            # Calculate weight w_nm = tau * n + xi * m
            weight_w_nm = tau * n + xi * m
            # Calculate |Anm| (the amplitude of the FrCHFM) and apply the weight
            wa_matrix[n, m] = weight_w_nm * abs(frchfm[n, m])

    return wa_matrix



In [19]:
def compute_frchfm(image, max_degree, t):
    """
    Compute Fractional Chebyshev-Fourier moments (FrCHFMs) of an input image.
    
    Parameters:
        image (2D numpy array): Grayscale image.
        max_degree (int): Maximum degree of the Chebyshev polynomial.
        t (float): Fractional parameter.
    
    Returns:
        frchfm (2D numpy array): The computed Fractional Chebyshev-Fourier moments.
    """
    # Convert image to polar coordinates
    polar_image, r, theta = convert_to_polar(image)
    
    # Fourier transform in angular direction (theta)
    F_image = np.fft.fft2(polar_image)
    F_image_shifted = np.fft.fftshift(F_image)
    
    # Initialize an array to store the FrCHFMs
    frchfm = np.zeros((max_degree + 1, max_degree + 1), dtype=np.complex128)
    
    # Loop over degrees for Chebyshev polynomials
    for n in range(max_degree + 1):
        for m in range(-max_degree, max_degree + 1):
            # Compute the radial basis function
            Rtn = radial_basis_function(n, t, r)
            
            # Angular Fourier component exp(-jmθ)
            angular_component = np.exp(-1j * m * theta)
            
            # Compute the FrCHFM using the formula
            frchfm[n, m] = np.sum(F_image_shifted * Rtn * angular_component * r)
    
    return frchfm



In [20]:
def generate_binary_sequence(wa_matrix):
    """
    Generate a binary sequence associated with the Weighted Amplitude (WA) matrix.
    
    Parameters:
        wa_matrix (2D numpy array): Weighted Amplitude matrix.
        
    Returns:
        binary_sequence (string): Binary sequence (string of 0s and 1s).
    """
    # Compute the average amplitude (threshold)
    threshold = np.mean(wa_matrix)
    
    # Generate binary sequence by comparing each WA element to the threshold
    binary_sequence = [(1 if wa_matrix[n, m] >= threshold else 0) 
                       for n in range(wa_matrix.shape[0]) 
                       for m in range(wa_matrix.shape[1])]
    binary_sequence_str = ''.join(map(str, binary_sequence))
    
    return binary_sequence_str


In [ ]:
# Load the grayscale image
image_path = "IMG_0612.jpg"
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

# Define the maximum degree for Chebyshev polynomials and the fractional parameter t
max_degree = 5  # Change depending on desired precision
fractional_parameter_t = 0.5  # Fractional parameter t

# Compute the FrCHFMs
frchfm = compute_frchfm(image, max_degree, fractional_parameter_t)
WAMatrix = compute_weighted_amplitude(frchfm)

# Display results
print("Fractional Chebyshev-Fourier Moments:")
print(frchfm)
print(generate_binary_sequence(WAMatrix))

# Visualize the magnitude of the FrCHFMs
plt.imshow(np.abs(frchfm), cmap='gray')
plt.colorbar()
plt.title('Fractional Chebyshev-Fourier Moments (Magnitude)')
plt.show()